Using this Notebook to test out code for getting the Lat / Lon for the uploaded file

In [1]:
import pandas
from geopy.geocoders import ArcGIS

geo = ArcGIS()

In [2]:
loc=geo.geocode("3666 21st St, San Francisco, CA 94114")

In [3]:
type(loc)

geopy.location.Location

In [4]:
print(loc.latitude)

37.7564496899993


In [5]:
print(loc.longitude)

-122.42936434915458


In [6]:
loc

Location(3666 21st St, San Francisco, California, 94114, (37.7564496899993, -122.42936434915458, 0.0))

In [7]:
print(loc)

3666 21st St, San Francisco, California, 94114


In [8]:
lon=loc.longitude
lat=loc.latitude
print(lat, lon)

37.7564496899993 -122.42936434915458


This suggests that we can iterate over the dataframe and input the latitude and longitude for each address in turn instead of having to create a column for the location object and then split out the latitude and longitude in turn.

The original supermarkets file has the address, city and state separated out. Should we build code in to accommodate that (producing a concatenated address), or give an error?
Build in the code to accommodate it (commas are not required in the address, just that it's a string - either in double quotes, or passed to the geocoder as a string variable)

In [9]:
geo.geocode("3666 21st St San Francisco CA 94114")

Location(3666 21st St, San Francisco, California, 94114, (37.7564496899993, -122.42936434915458, 0.0))

In [10]:
a="3666 21st St San Francisco CA 94114"
geo.geocode(a)

Location(3666 21st St, San Francisco, California, 94114, (37.7564496899993, -122.42936434915458, 0.0))

In [11]:
df=pandas.read_csv("supermarkets.csv")
df

,ID,Address,City,State,Country,Name,Employees
0,1,3666 21st St,San Francisco,CA 94114,USA,Madeira,8
1,2,735 Dolores St,San Francisco,CA 94119,USA,Bready Shop,15
2,3,332 Hill St,San Francisco,California 94114,USA,Super River,25
3,4,3995 23rd St,San Francisco,CA 94114,USA,Ben's Shop,10
4,5,1056 Sanchez St,San Francisco,California,USA,Sanchez,12
5,6,551 Alvarado St,San Francisco,CA 94114,USA,Richvalley,20


In [12]:
df["Address"]=df["Address"] + " " + df["City"] + " " + df["State"] + " " + df["Country"]
df

,ID,Address,City,State,Country,Name,Employees
0,1,3666 21st St San Francisco CA 94114 USA,San Francisco,CA 94114,USA,Madeira,8
1,2,735 Dolores St San Francisco CA 94119 USA,San Francisco,CA 94119,USA,Bready Shop,15
2,3,332 Hill St San Francisco California 94114 USA,San Francisco,California 94114,USA,Super River,25
3,4,3995 23rd St San Francisco CA 94114 USA,San Francisco,CA 94114,USA,Ben's Shop,10
4,5,1056 Sanchez St San Francisco California USA,San Francisco,California,USA,Sanchez,12
5,6,551 Alvarado St San Francisco CA 94114 USA,San Francisco,CA 94114,USA,Richvalley,20


In [13]:
geo.geocode(a).latitude

37.7564496899993

So it is possible to return the latitude or longitude directly on the address, using the format above.

In [14]:
df["Latitude"] = df["Address"].apply(lambda lat: geo.geocode(lat).latitude if lat !=None else None)
df["Longitude"] = df["Address"].apply(lambda lon: geo.geocode(lon).longitude if lon !=None else None)
df

,ID,Address,City,State,Country,Name,Employees,Latitude,Longitude
0,1,3666 21st St San Francisco CA 94114 USA,San Francisco,CA 94114,USA,Madeira,8,37.756450,-122.429364
1,2,735 Dolores St San Francisco CA 94119 USA,San Francisco,CA 94119,USA,Bready Shop,15,37.757796,-122.425597
2,3,332 Hill St San Francisco California 94114 USA,San Francisco,California 94114,USA,Super River,25,37.755648,-122.428803
3,4,3995 23rd St San Francisco CA 94114 USA,San Francisco,CA 94114,USA,Ben's Shop,10,37.752991,-122.431702
4,5,1056 Sanchez St San Francisco California USA,San Francisco,California,USA,Sanchez,12,37.752129,-122.430028
5,6,551 Alvarado St San Francisco CA 94114 USA,San Francisco,CA 94114,USA,Richvalley,20,37.753719,-122.433240


This seems to be a good one-step way to return the lat / lon values for the address

In [15]:
df1 = df.drop(columns = ["City", "State", "Country"])
df1

,ID,Address,Name,Employees,Latitude,Longitude
0,1,3666 21st St San Francisco CA 94114 USA,Madeira,8,37.756450,-122.429364
1,2,735 Dolores St San Francisco CA 94119 USA,Bready Shop,15,37.757796,-122.425597
2,3,332 Hill St San Francisco California 94114 USA,Super River,25,37.755648,-122.428803
3,4,3995 23rd St San Francisco CA 94114 USA,Ben's Shop,10,37.752991,-122.431702
4,5,1056 Sanchez St San Francisco California USA,Sanchez,12,37.752129,-122.430028
5,6,551 Alvarado St San Francisco CA 94114 USA,Richvalley,20,37.753719,-122.433240


This is a way to create the output file, assuming that the input file has a split address field.

In [16]:
b="3666 21st St"
geo.geocode(b)

Location(Travessa da Santa 21, 4740-451, Santa, Esposende, Braga, (41.60956056023514, -8.736948802464127, 0.0))

Absolutely need the city, state / county / province and country in order for this to work properly.
*Initially* let's get this working based on a perfect file, then introduce an enhancement to enable the user to select the fields to combine if they have a split address in the file.

In [17]:
print(df1.to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>ID</th>
      <th>Address</th>
      <th>Name</th>
      <th>Employees</th>
      <th>Latitude</th>
      <th>Longitude</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1</td>
      <td>3666 21st St San Francisco CA 94114 USA</td>
      <td>Madeira</td>
      <td>8</td>
      <td>37.756450</td>
      <td>-122.429364</td>
    </tr>
    <tr>
      <th>1</th>
      <td>2</td>
      <td>735 Dolores St San Francisco CA 94119 USA</td>
      <td>Bready Shop</td>
      <td>15</td>
      <td>37.757796</td>
      <td>-122.425597</td>
    </tr>
    <tr>
      <th>2</th>
      <td>3</td>
      <td>332 Hill St San Francisco California 94114 USA</td>
      <td>Super River</td>
      <td>25</td>
      <td>37.755648</td>
      <td>-122.428803</td>
    </tr>
    <tr>
      <th>3</th>
      <td>4</td>
      <td>3995 23rd St San Francisco CA 94114 USA</td>
      <td>Ben's Sh

In [19]:
dl=df1.to_csv(index=False)
dl

"ID,Address,Name,Employees,Latitude,Longitude\r\n1,3666 21st St San Francisco CA 94114 USA,Madeira,8,37.7564496899993,-122.42936434915458\r\n2,735 Dolores St San Francisco CA 94119 USA,Bready Shop,15,37.7577961290611,-122.42559668301706\r\n3,332 Hill St San Francisco California 94114 USA,Super River,25,37.75564836499496,-122.4288026611172\r\n4,3995 23rd St San Francisco CA 94114 USA,Ben's Shop,10,37.752990821253434,-122.43170235858965\r\n5,1056 Sanchez St San Francisco California USA,Sanchez,12,37.752129,-122.43002849999999\r\n6,551 Alvarado St San Francisco CA 94114 USA,Richvalley,20,37.75371876406616,-122.43323998612154\r\n"

In [23]:
df1.to_csv("download.csv", index=False)

In [24]:
df2=pandas.read_csv("supermarkets v2.csv")
df2

,ID,address,Name,Employees
0,1,3666 21st St San Francisco CA 94114 USA,Madeira,8
1,2,735 Dolores St San Francisco CA 94119 USA,Bready Shop,15
2,3,332 Hill St San Francisco California 94114 USA,Super River,25
3,4,3995 23rd St San Francisco CA 94114 USA,Ben's Shop,10
4,5,1056 Sanchez St San Francisco California USA,Sanchez,12
5,6,551 Alvarado St San Francisco CA 94114 USA,Richvalley,20


In [25]:
df2.columns

Index(['ID', 'address', 'Name', 'Employees'], dtype='object')

To rename a column in a dataframe, can use the .rename function and pass a dictionary containing the old and new names of the columns to be changed.
*Should* be able to iterate over the original list of column names to find 'address' in there, check if it's in title / lower case format and then switch it to the required variant.
OR
have the lat / lon population functions duplicated to work for both variants, but just use whichever one is required for the current file format.

In [27]:
df2.columns[1]

'address'

In [28]:
df1.columns[1]

'Address'

In [29]:
for c in df2.columns:
    print(c)

ID
address
Name
Employees


In [30]:
for c in df2.columns:
    if c == "address":
        df2.rename(columns={"address" : "Address"}, inplace=True)
df2

,ID,Address,Name,Employees
0,1,3666 21st St San Francisco CA 94114 USA,Madeira,8
1,2,735 Dolores St San Francisco CA 94119 USA,Bready Shop,15
2,3,332 Hill St San Francisco California 94114 USA,Super River,25
3,4,3995 23rd St San Francisco CA 94114 USA,Ben's Shop,10
4,5,1056 Sanchez St San Francisco California USA,Sanchez,12
5,6,551 Alvarado St San Francisco CA 94114 USA,Richvalley,20


This looks to be a simple way of accounting for and changing "address" to "Address" as a column header. It's likely not the most elegant solution, but it works.

In [36]:
def add_check(frame):
    count = 1
    for c in frame.columns:
            if c == "address":
                frame.rename(columns={"address" : "Address"}, inplace=True)
                action = "Renamed address to Address"
                results(action)
            elif c == "Address":
                action = "No action required"
                results(action)
            else:
                if count == len(frame.columns):
                    action = "Please ensure you have a column titled 'Address' or 'address' in your upload file."
                    results(action)
            count += 1

def results(action):
    print(action)

This needs updating to include a flag to show when the address and Address rules have been tripped on a dataframe. These flags (this flag?) can then be checked when the final 'else' condition is met.

In [37]:
dft1=pandas.read_csv("supermarkets v1.csv")
dft2=pandas.read_csv("supermarkets v2.csv")
dft3=pandas.read_csv("supermarkets fail test.csv")

In [38]:
add_check(dft1)
add_check(dft2)
add_check(dft3)

No action required
Please ensure you have a column titled 'Address' or 'address' in your upload file.
Renamed address to Address
Please ensure you have a column titled 'Address' or 'address' in your upload file.
Please ensure you have a column titled 'Address' or 'address' in your upload file.


In [11]:
dft1

,ID,Address,Name,Employees
0,1,3666 21st St San Francisco CA 94114 USA,Madeira,8
1,2,735 Dolores St San Francisco CA 94119 USA,Bready Shop,15
2,3,332 Hill St San Francisco California 94114 USA,Super River,25
3,4,3995 23rd St San Francisco CA 94114 USA,Ben's Shop,10
4,5,1056 Sanchez St San Francisco California USA,Sanchez,12
5,6,551 Alvarado St San Francisco CA 94114 USA,Richvalley,20


In [12]:
dft2

,ID,Address,Name,Employees
0,1,3666 21st St San Francisco CA 94114 USA,Madeira,8
1,2,735 Dolores St San Francisco CA 94119 USA,Bready Shop,15
2,3,332 Hill St San Francisco California 94114 USA,Super River,25
3,4,3995 23rd St San Francisco CA 94114 USA,Ben's Shop,10
4,5,1056 Sanchez St San Francisco California USA,Sanchez,12
5,6,551 Alvarado St San Francisco CA 94114 USA,Richvalley,20


Somehow, the address column is still being changed as well as producing the 'failure' text...
Ah, the failures are being flagged for the ID, Name and Employees columns (and Headdress in the fail test file).
Need to work out how to stop the failure being flagged until *all* of the columns have been checked.

In [15]:
len(dft2.columns)

4

In [46]:
dft1=pandas.read_csv("supermarkets v1.csv")
dft2=pandas.read_csv("supermarkets v2.csv")
dft3=pandas.read_csv("supermarkets fail test.csv")

In [3]:
print(dft1.filter(regex="address|Address").columns)
print(dft2.filter(regex="address|Address").columns)
print(dft3.filter(regex="address|Address").columns)

Index(['Address'], dtype='object')
Index(['address'], dtype='object')
Index(['Headdress'], dtype='object')


In [4]:
print(dft1.filter(like="address").columns)
print(dft2.filter(like="address").columns)
print(dft3.filter(like="address").columns)

Index([], dtype='object')
Index(['address'], dtype='object')
Index(['Headdress'], dtype='object')


In [5]:
print(dft1.filter("Address" or "address").columns)

Index([], dtype='object')


In [17]:
def address_check(frame):
    add_cols = [col for col in frame.columns if col == "Address" or col == "address"]
    print(add_cols)

In [18]:
address_check(dft1)
address_check(dft2)
address_check(dft3)

['Address']
['address']
[]


In [47]:
def address_check(frame):
    add_cols = [col for col in frame.columns if col == "Address" or col == "address"]
    # check to see if the list contains any results
    if add_cols:
        # if it does, check for a lower case address and change to title case
        if add_cols[0] == "address":
            frame.rename(columns={"address" : "Address"}, inplace=True)
            print("address column found and renamed to Address")
        # otherwise there will be a title case Address
        else:
            print("Address column found successfully")
    # if the list is empty, return an error to the user
    else:
        print("No address column found in file")

This is the method to use. Remove the print statements, and the 'else' statement where the list is not empty.

In [48]:
address_check(dft1)
address_check(dft2)
address_check(dft3)

Address column found successfully
address column found and renamed to Address
No address column found in file


In [23]:
dft1

,ID,Address,Name,Employees
0,1,3666 21st St San Francisco CA 94114 USA,Madeira,8
1,2,735 Dolores St San Francisco CA 94119 USA,Bready Shop,15
2,3,332 Hill St San Francisco California 94114 USA,Super River,25
3,4,3995 23rd St San Francisco CA 94114 USA,Ben's Shop,10
4,5,1056 Sanchez St San Francisco California USA,Sanchez,12
5,6,551 Alvarado St San Francisco CA 94114 USA,Richvalley,20
